In [5]:
import torch
print("Torch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())

from ultralytics import YOLO
print("YOLO Imported Successfully!")


Torch Version: 2.6.0+cu118
CUDA Available: False
YOLO Imported Successfully!


In [2]:
import cv2
import numpy as np
import pandas as pd
from pyzbar.pyzbar import decode
from ultralytics import YOLO
import os

# Load YOLO models
object_model_path = r"C:\Users\abhin\Desktop\computer_vision\qr_code\yolov8m.pt"  # Object detection
barcode_model_path = r"C:\Users\abhin\Desktop\computer_vision\qr_code\best.pt"  # Barcode detection

object_model = YOLO(object_model_path)  # Object detection model
barcode_model = YOLO(barcode_model_path)  # Barcode detection model

# CSV file path for storing detected barcode data
csv_file = r"C:\Users\abhin\Desktop\computer_vision\qr_code\detected_barcodes.csv"

# Load existing barcodes from CSV (only for the current image)
existing_barcodes = set()

def load_existing_barcodes():
    """Loads existing barcodes from the CSV for the current image processing session."""
    global existing_barcodes
    existing_barcodes.clear()
    
    if os.path.exists(csv_file):
        try:
            df = pd.read_csv(csv_file)
            existing_barcodes = set(df["Barcode Data"].astype(str).str.lower())  # Normalize
        except pd.errors.EmptyDataError:
            pass  # Handle empty CSV case

def save_to_csv(barcode_data, barcode_type):
    """Saves barcode data to CSV only if it's not a duplicate for this image."""
    barcode_data_lower = barcode_data.lower()

    if barcode_data_lower in existing_barcodes:
        print(f"ℹ️ Skipping duplicate for this image: {barcode_data} ({barcode_type})")
        return

    # Save new barcode
    new_data = pd.DataFrame([[barcode_data, barcode_type]], columns=["Barcode Data", "Type"])
    new_data.to_csv(csv_file, mode='a', header=not os.path.exists(csv_file), index=False)

    # Update local set to prevent duplicates for this image
    existing_barcodes.add(barcode_data_lower)

    print(f"✅ Saved: {barcode_data} ({barcode_type})")

# Load the image
image_path = r"C:\Users\abhin\Desktop\computer_vision\qr_code\WhatsApp Image 2025-03-24 at 23.32.12_0911c97f.jpg"
image = cv2.imread(image_path)

if image is None:
    print("❌ Error: Could not load image")
    exit()

# Load existing barcodes for this image
load_existing_barcodes()

# Detect objects in the image
object_results = object_model(image)

# Draw bounding boxes for detected objects
for object_result in object_results:
    if hasattr(object_result, 'boxes') and object_result.boxes is not None:
        for box in object_result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box for objects

# 📌 **Decode barcodes only ONCE per image**
barcode_results = barcode_model(image)

if hasattr(barcode_results[0], 'boxes') and barcode_results[0].boxes is not None:
    for barcode_box in barcode_results[0].boxes:
        bx1, by1, bx2, by2 = map(int, barcode_box.xyxy[0].tolist())
        cv2.rectangle(image, (bx1, by1), (bx2, by2), (0, 0, 255), 2)  # Red box for barcodes

# **Decode and save barcode data only once**
decoded_objects = decode(image)
for obj in decoded_objects:
    barcode_data = obj.data.decode("utf-8")
    barcode_type = obj.type

    # Check for duplicates before storing
    save_to_csv(barcode_data, barcode_type)

    # Place text on barcode
    text_x, text_y = max(bx1, 10), max(by1 - 10, 20)
    cv2.putText(image, barcode_data, (text_x, text_y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

# Show final image without zooming
cv2.imshow("Object & Barcode Detection", cv2.resize(image, (600, 600)))  # Resize for display
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 640x480 1 toilet, 1 book, 508.8ms
Speed: 4.0ms preprocess, 508.8ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 barcode, 182.5ms
Speed: 4.0ms preprocess, 182.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)
ℹ️ Skipping duplicate for this image: 9788183227438 (EAN13)
